<a href="https://colab.research.google.com/github/djurjo/DABI/blob/main/PSO-project/islands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numba
from numba import jitclass, types, typed, njit, prange
from numba import int32, float32
import random
import numpy as np
import time
import csv
import math

In [ ]:
### All of this is copied from swarn_numba.ipynb

@njit
def create_random_array(size, list_intervals):
    r = np.zeros(size, dtype = np.float32)
    for i in range(0, size):
      b = random.random() # We use random randoms instead of np.randoms
      interval = list_intervals[i]
      b = (interval[1] - interval[0]) * b + interval[0]
      r[i] += b
    return r

# Some parallelized functions to operate with vectors
@njit
def add(v1, v2):
  r = np.zeros(v1.size, dtype = np.float32)
  if v1.size == v2.size:
    for i in range(0, v1.size):
      r[i] = v1[i] + v2[i]
  return r

@njit
def escalar(v1, val):
  r = np.zeros(v1.size, dtype = np.float32)
  for i in range(v1.size):
    r[i] = v1[i]* val
  return r

@njit
def sub(v1, v2):
  r = np.zeros(v1.size, dtype = np.float32)
  if v1.size == v2.size:
    for i in range(0, v1.size):
      r[i] = v1[i] - v2[i]
  return r

def create_swarn(search_space,dimension, B):
  swarn = []
  for i in prange(B):
    position = create_random_array(dimension, search_space)
    direction = create_random_array(dimension, search_space)
    b = np.vstack((position, direction, position))
    swarn.append(b)
  t_swarn = typed.List()    
  [t_swarn.append(x) for x in swarn]   
  return t_swarn

def update_swarn_vector(swarn, best_global_postion, cog, soc, inertia, r1, r2):
  for i in prange(len(swarn)):
    bird = swarn[i]
    s1 = escalar(bird[1], inertia)
    s2 = escalar(sub(bird[2], bird[0]) , cog*r1)
    s3 = escalar(sub(best_global_postion, bird[0]) , soc*r2)
    swarn[i][1] = add(add(s1, s2), s3)

@njit
def in_search_space(current_pos, search_space):
    for i in range(len(search_space)):
        if (search_space[i][0] <= current_pos[i]) and (current_pos[i] <= search_space[i][1]):
           pass
        else:
            if (abs(search_space[i][0] - current_pos[i]) < abs(search_space[i][1] - current_pos[i])):
                current_pos[i] = current_pos[i] % search_space[i][0]
            else:
                current_pos[i] = current_pos[i] % search_space[i][1]
    return current_pos


@njit
def move_swarn(swarn, search_space):
  for i in range(len(swarn)):
    bird = swarn[i]
    n_pos = add(bird[0], bird[1])
    swarn[i][0] = in_search_space(n_pos, search_space)

def update_best_local(swarn, f):
  for i in prange(len(swarn)):
    bird = swarn[i]
    if f(bird[0]) < f(bird[2]):
      swarn[i][2] = bird[0]
@njit
def take_best_global(swarn, f):
  b_g_p = swarn[0][2]
  b_g_v = f(b_g_p)
  for i in prange(len(swarn)):
    bird = swarn[i]
    if f(bird[2]) < b_g_v :
      b_g_p = bird[2]
      b_g_v = f(bird[2])
    else:
      pass
  return (b_g_p, b_g_v)




In [ ]:
@njit
def move_bird(bird, search_space):
    n_pos = add(bird[0], bird[1])
    bird[0] = in_search_space(n_pos, search_space)
    return bird
@njit
def bird_update_local(bird, f):
    if f(bird[0]) < f(bird[2]):
        bird[2] = bird[0]
    return bird
@njit
def update_bird(bird, best_global_postion, cog, soc, inertia, r1, r2):
    s1 = escalar(bird[1], inertia)
    s2 = escalar(sub(bird[2], bird[0]) , cog*r1)
    s3 = escalar(sub(best_global_postion, bird[0]) , soc*r2)
    bird[1] = add(add(s1, s2), s3)
    return bird
@njit
def bird_upd(bird, b_g_p, cog, soc, inertia ,r1, r2, search_space, f):
    bird = update_bird(bird, b_g_p, cog, soc, inertia ,r1, r2)
    bird = move_bird(bird, search_space)
    bird = bird_update_local(bird, f)
    return bird

In [ ]:

### Here we will put some global constans

cog = 1.8  # Cognitive importance
soc = 2.3   # Social importance
inertia = 0.8

# Two random values
r1 = np.random.random() 
r2 = np.random.random()

# About the search space:

dimensions = 5
search_space = [(-30, 30)]*dimensions

t_space = typed.List()
[t_space.append(x) for x in search_space]

# Number of birds
B = 100

# Number of iterations

iterations = 1000

runs = 10

In [ ]:
@njit
def swarn_upd(swarn, b_g_p, cog, soc, inertia ,r1, r2, search_space, f):
  for j in prange(0, len(swarn)):
    swarn[j] = bird_upd(swarn[j], b_g_p, cog, soc, inertia ,r1, r2, search_space, f)
  return swarn

@njit(parallel = True)
def islands_upd(swarn_list, b_g_p, cog, soc, inertia ,r1, r2, search_space, f):
  for j in prange(0, len(swarn_list)):
    t = take_best_global(swarn_list[j], f)
    swarn_list[j] = swarn_upd(swarn_list[j], t[0], cog, soc, inertia ,r1, r2, search_space, f)
  return swarn_list

@njit
def run_pso(iterations, B, space, dimensions, target, cog, soc, inertia, r1, r2):
  swarn = create_swarn(space, dimensions,B)
  t = take_best_global(swarn, target)
  b_g_p = t[0]
  b_g_v = t[1]
  for it in range(1, iterations + 1):
    swarn = swarn_upd(swarn, b_g_p, cog, soc, inertia ,r1, r2, space, target) ## In each iteration share the info.
    t = take_best_global(swarn, target)
    b_g_p = t[0]
    b_g_v = t[1]
  return (b_g_v, b_g_p, swarn)

@njit
def best_global_islands(swarn_list, target):
  best = (None, 10**6)
  for k in range(len(swarn_list)):
    t = take_best_global(swarn_list[k], target)
    if t[1] < best[1]:
      best = t
  return best

def islands_pso(iterations, B, space, dimensions, target, cog, soc, inertia, r1, r2, n_islands):
  birds_islands = int(B/n_islands)
  swarn_list = typed.List()
  for k in range(n_islands):
    swarn_list.append(create_swarn(space, dimensions,birds_islands))
  t = best_global_islands(swarn_list, target)
  b_g_p = t[0]
  b_g_v = t[1]  
  for it in range(0, iterations):
    swarn_list = islands_upd(swarn_list, b_g_p, cog, soc, inertia ,r1, r2, space, target)
    if it % 25 == 0:
      t = best_global_islands(swarn_list, target)
      b_g_p = t[0] 
      b_g_v = t[1]
  t = best_global_islands(swarn_list, target)
  b_g_p = t[0] 
  b_g_v = t[1]
  return b_g_v

     






In [ ]:
#### Global variables
runs = 1

cog = 1.7  # Cognitive importance
soc = 2.3   # Social importance
inertia = 0.6

# Two random values that will change each time
r1 = random.random() 
r2 = random.random()

# Number of birds
B = 100

# Number of iterations

iterations = 1000

pairs = [(100, 1000), (200, 1000), (500, 1000), (100, 5000)]# (500, 5000)]


In [ ]:
#### TEST FUNCTIONS ####

@njit
def sphere(x):
  s = 0
  for i in prange(len(x)):
    s += x[i]**2
  return s

def sphere2(x):
  s = 0
  x = list(x)
  for i in range(len(x)):
    s += x[i]**2
  return s


@njit
def rosenbrock(x):
  s = 0
  for i in prange(len(x)-1):
    s += 100*(x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
  return s

def rosenbrock2(x):
  s = 0
  x = list(x)
  for i in range(len(x)-1):
    s += 100*(x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
  return s

@njit 
def styblinski(x):
  s = 0
  for i in prange(len(x)):
    s += (x[i]**4) - 16*(x[i]**2) + 5*x[i]
  return s/2

def styblinski2(x):
  s = 0
  x = list(x)
  for i in range(len(x)):
    s += (x[i]**4) - 16*(x[i]**2) + 5*x[i]
  return s/2

def minStyb(dimension):
  sol = [-2.903534]*dimension
  solsty = typed.List()
  [solsty.append(x) for x in sol]
  return styblinski2(solsty)

### min between -39.16617*n < f(-2.903534, ... , -2.903534)<-39.16616*n 

@njit
def f8(x):
  s = 0
  for i in prange(len(x)):
    s += (- x[i]) * (math.sin(math.sqrt(abs(x[i]))))
  return s

def f82(x):
  s = 0
  x = list(x)
  for i in range(len(x)):
    s += (- x[i]) * (math.sin(math.sqrt(abs(x[i]))))
  return s

## min  [-500, 500] is -12569.5

@njit 
def f2(x):
  s = 0
  p = 0
  for i in range(len(x)):
    s += abs(x[i])
    p *= abs(x[i])
  return s + p

def f22(x):
  s = 0
  p = 0
  x = list(x)
  for i in range(len(x)):
    s += abs(x[i])
    p *= abs(x[i])
  return s + p

@njit
def u(x, a, k, m):
  if x > a:
    return k*((x - a)**m)
  elif -a <= x and x <= a:
    return 0
  else:
    return k*((- x - a)**m)

def y(x):
  return 1 + (1/4)*(x + 1)

@njit
def f13(x):
  s1 = 0
  s2 = 0
  for i in range(len(x)):
    if i < len(x) - 1:
      s1 += ((x[i] - 1)**2) * (1 + (math.sin(3 * math.pi * x[i+1]))**2) 
    s2 += u(x[i], 5, 100, 4)
  return 0.1*((math.sin(3 * math.pi * x[1]))**2 + s1 + (x[len(x) - 1] - 1)*(1 + (math.sin(2 * math.pi * x[len(x) - 1]))**2)) + s2

def f132(x):
  s1 = 0
  s2 = 0
  x = list(x)
  for i in range(len(x)):
    if i < len(x) - 1:
      s1 += ((x[i] - 1)**2) * (1 + (math.sin(3 * math.pi * x[i+1]))**2) 
    s2 += u(x[i], 5, 100, 4)
  return 0.1*((math.sin(3 * math.pi * x[1]))**2 + s1 + (x[len(x) - 1] - 1)*(1 + (math.sin(2 * math.pi * x[len(x) - 1]))**2)) + s2



target_functions = { # id : (function, space, global_min, function2)
    1 : (sphere, [(-500, 500)]*5, 0, sphere2),
    2 : (rosenbrock, [(-100, 100)]*5, 0, rosenbrock2),
    3 : (styblinski, [(-5, 5)]*5, minStyb(5), styblinski2), 
    4 : (f8, [(-500, 500)]*5, -12569.5, f82),
    5 : (f2, [(-10, 10)]*5, 0, f22),
    6 : (f13, [(-50, 50)]*5, 0, f132)
}
names = {
    1 : 'Sphere',
    2 : 'Rosenbrock',
    3 : 'Styblinski', 
    4 : 'f8', 
    5 : 'f2',
    6 : 'f13'
}
n_islands = [2, 5, 10, 100]

In [ ]:


#### This cell runs the experiments and record all the data in a csv file.

### We run the pso 40 times for each function and type (parallel or not) 
### and we record the mean time per run and the absolute mean error.

### First we are going to run all the functions and the pso in order to initialize the @njit functions

p = typed.List()
[p.append(x) for x in [0, 1 , 0, 1, -1, 1, 0, 1]]

for k in target_functions:
    x = target_functions[k][0](p)

t_sp = typed.List()
t_sp.append((-1, 1))
x = islands_pso(10, 10, t_sp, len(t_sp),sphere, cog, soc, inertia, r1, r2, 10)


print('Experiment starts: ')

with open('experiments_results.csv', mode='w') as experiment:
  exp = csv.writer(experiment, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  exp.writerow(['N. Birds', 'Iterations', 'Run', 'Target function', 'N Islands', 'Error', 'Time (s)'])

  for (iterations, B) in pairs:
    for n in n_islands:
      for key in target_functions.keys():
        (target, space, g_min, target2) = target_functions[key]
        t_space = typed.List()
        for x in space:
          t_space.append(x)
        
        total_time_par = 0
        total_error_par = 0

        for run in range(runs):
          
          time_par = time.time()
          solution = islands_pso(iterations, B, t_space, len(t_space),target, cog, soc, inertia, r1, r2, n)
          time_t_par = time.time() - time_par
          

          error_par = abs(g_min - solution)
          total_time_par = total_time_par + time_t_par
          total_error_par = total_error_par + solution
          

          exp.writerow([B, iterations, str(run + 1), names[key], str(n),error_par, time_t_par])

        
          r1 = random.random()
          r2 = random.random()

